In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.get_device_name(0))

Using device: cuda
NVIDIA GeForce RTX 3070 Laptop GPU


In [2]:
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
import json
from trl import SFTTrainer, SFTConfig

c:\Users\lohki\Downloads\prototype\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
df = pd.read_csv("dataset/synthetic_data.csv")

In [4]:
qa_data = []
for _, row in df.iterrows():
    qa_data.append({"question": row["question"], "answer": row["answer"]})

with open("dataset/synthetic_data.json", "w", encoding="utf-8") as f:
    json.dump(qa_data, f, indent=4)

In [5]:
data = pd.read_json("dataset/synthetic_data.json")

In [6]:
max_seq_length = 2048
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA GeForce RTX 3070 Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407, 
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [8]:
data_prompt = """### Instruction: Answer the question using the provided data.
If no relevant data is available, provide a response using your own knowledge or say: **"I don't know."**

### Context:
N/A

### Question:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token 

def formatting_prompt(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []

    for question, answer in zip(questions, answers):
        text = data_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)

    return { "text": texts, }


In [9]:
training_data = Dataset.from_pandas(data)
training_data = training_data.map(formatting_prompt, batched=True)

Map: 100%|██████████| 20/20 [00:00<00:00, 6469.20 examples/s]


In [10]:
print(training_data[0])

{'question': 'Which of my items had the highest total revenue last month, and how does it compare to the previous month?', 'answer': 'Last month, your top-selling item was Spicy Chicken Rice, generating RM4,250 in revenue  a 12% increase from RM3,780 the previous month. This growth suggests strong repeat demand or successful recent promotions.', 'text': '### Instruction: Answer the question using the provided data.\nIf no relevant data is available, provide a response using your own knowledge or say: **"I don\'t know."**\n\n### Context:\nN/A\n\n### Question:\nWhich of my items had the highest total revenue last month, and how does it compare to the previous month?\n\n### Answer:\nLast month, your top-selling item was Spicy Chicken Rice, generating RM4,250 in revenue  a 12% increase from RM3,780 the previous month. This growth suggests strong repeat demand or successful recent promotions.<|end_of_text|>'}


In [11]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_num_proc=1,
    args = SFTConfig(
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        packing=True,
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=10,
        fp16=False,
        bf16=True,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
        report_to = "none",
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"]: 100%|██████████| 20/20 [00:00<00:00, 1257.32 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 10 | Total steps = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.303500
2,3.270500
3,3.286300
4,3.261900
5,3.157000
6,3.093200
7,2.785800
8,2.586600
9,2.265600
10,1.986900


TrainOutput(global_step=10, training_loss=2.8997328519821166, metrics={'train_runtime': 8.5467, 'train_samples_per_second': 23.401, 'train_steps_per_second': 1.17, 'total_flos': 73807746760704.0, 'train_loss': 2.8997328519821166})

In [13]:
text = "What are the top-selling items in the last 30 days?"

In [14]:
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]
print("Answer of the question is:", answer)

Answer of the question is: <|begin_of_text|>### Instruction: Answer the question using the provided data.
If no relevant data is available, provide a response using your own knowledge or say: **"I don't know."**

### Context:
N/A

### Question:
What are the top-selling items in the last 30 days?

### Answer:
The top-selling items in the last 30 days are:

* **"N/A":** N/A

### Additional Context:
No relevant data is available.<|end_of_text|>


In [15]:
model.save_pretrained("model/finetuned_llama3.2")
tokenizer.save_pretrained("model/finetuned_llama3.2")
model.config.save_pretrained("model/finetuned_llama3.2")